<a href="https://colab.research.google.com/github/harsh16kh/stock_price_analysis/blob/main/backtrading_on_btc_using_rsi_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving BTCUSDT-20210101-20210102-1m.csv to BTCUSDT-20210101-20210102-1m.csv


In [ ]:
pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install binance.client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement binance.client (from versions: none)
ERROR: No matching distribution found for binance.client


In [ ]:
python -m pip uninstall python-binance



SyntaxError: ignored

In [ ]:
!pip install python-binance==0.7.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 8.0 MB/s 
     |████████████████████████████████| 396 kB 63.8 MB/s 
     |████████████████████████████████| 4.0 MB 39.6 MB/s 
     |████████████████████████████████| 288 kB 75.6 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
     |████████████████████████████████| 251 kB 34.7 MB/s 
  Created wheel for autobahn: filename=autobahn-22.4.2-cp37-cp37m-linux_x86_64.whl size=580338 sha256=6261f4683092e64ac89ac71fa137fee54a4fbc477dcff4b6ee2c1c465e90220a
  Stored in directory: /root/.cache/pip/wheels/ec/0b/fc/7cf072cb66e08c9365e03420a6447aa45da6f70dd4b08b223c
Successfully built autobahn


In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
 !pip install --upgrade python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 112 kB 11.0 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 61.7 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 51.0 MB/s 
  Attempting uninstall: python-binance
    Found existing installation: python-binance 0.7.5
    Uninstalling python-binance-0.7.5:
      Successfully uninstalled python-binance-0.7.5


In [ ]:
import config, csv, binance
from binance import Client

api_key = secret(Dont share)
api_secret = secret(Dont share)

client = Client(api_key, api_secret)

dataStart = '1 Jan, 2021'
dataEnd = '2 Jan, 2021'
datafile = 'BTCUSDT-20210101-20210102-1m.csv'
csvfile = open(datafile, 'w', newline='')
candlestick_writer = csv.writer(csvfile, delimiter=',')

candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, dataStart, dataEnd)

for candlestick in candlesticks:
    candlestick[0] = candlestick[0] / 1000 # divide timestamp to ignore miliseconds
    candlestick_writer.writerow(candlestick)

csvfile.close()

print(datafile, len(candlesticks))

BTCUSDT-20210101-20210102-1m.csv 1441


In [ ]:
candlesticks[0]

[1609459200.0,
 '28923.63000000',
 '28961.66000000',
 '28913.12000000',
 '28961.66000000',
 '27.45703200',
 1609459259999,
 '794382.04398665',
 1292,
 '16.77719500',
 '485390.82682460',
 '0']

In [ ]:
!pip install backtrader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 8.0 MB/s 


In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import backtrader as bt # Import the backtrader platfor


class RSIStrategy(bt.Strategy):

    params = (
        ('verbose', False),
        ('maperiod', None),
        ('quantity', None),
        ('upper', 70),         # upper threshold
        ('lower', 30),         # lower threshold
        ('stopLoss', 0.00)          # stop loss %
    )


    def __init__(self):

        self.dataclose = self.datas[0].close
        self.order = None
        self.order_stopLoss = None
        self.buyprice = None
        self.buycomm = None
        self.amount = None

        # Add a MovingAverageSimple indicator
        self.rsi = bt.indicators.RSI_SMA(self.datas[0], period=self.params.maperiod)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

                if self.params.verbose:
                    print('BOUGHT @price: {:.2f} {}'.format(order.executed.price, bt.num2date(order.executed.dt)))
                if self.params.stopLoss:
                    if self.params.stopLoss > 0:
                        stop_price = order.executed.price * (1 - self.params.stopLoss)
                        self.order_stopLoss = self.sell(exectype=bt.Order.Stop, price=stop_price)
                        if self.params.verbose:
                            print('  STOP @price: {:.2f}'.format(stop_price))
                    else:
                        # trailing stop specified % under executed price
                        self.order_stopLoss = self.sell(exectype=bt.Order.StopTrail, trailpercent=0-self.params.stopLoss)
                        self.order_stopLoss.addinfo(ordername="STOPLONG")
                        if self.params.verbose:
                            print('  STOP TRAILING')

            else:
                if not self.position:  # we left the market
                    self.broker.cancel(self.order_stopLoss)
                    self.order_stopLoss = None
                    if self.params.verbose:
                        print('SOLD @price: {:.2f} cost: {:.2f} comm: {:.2f} {}'.format(order.executed.price, order.executed.value, order.executed.comm, bt.num2date(order.executed.dt)))

        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        if self.params.verbose:
            print('PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
            print('_______________________________________________')



    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.rsi < self.params.lower:

                # Keep track of the created order to avoid a 2nd order
                self.amount = (self.broker.getvalue() * self.params.quantity) / self.dataclose[0]
                self.order = self.buy()

        else:
            # Already in the market ... we might sell
            if self.rsi > self.params.upper:

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()



# ______________________ End Strategy Classes



def timeFrame(datapath):
    """
    Select the write compression and timeframe.
    """
    sepdatapath = datapath[5:-4].split(sep='-') # ignore name file 'data/' and '.csv'
    tf = sepdatapath[3]

    if tf == '1m':
        compression = 1
        timeframe = bt.TimeFrame.Minutes
    else:
        print('dataframe not recognized')
        exit()

    return compression, timeframe


def getWinLoss(analyzer):
    return analyzer.won.total, analyzer.lost.total, analyzer.pnl.net.total


def getSQN(analyzer):
    return round(analyzer.sqn,2)



def runbacktest(datapath, start, end, period, strategy, \
                upper=70, lower=30, commission_val=None, portofolio=10000.0, stake_val=1, quantity=0.01, stopLoss=0.0, plt=False):

    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=stake_val) # Multiply the stake by X

    cerebro.broker.setcash(portofolio)

    if commission_val:
        cerebro.broker.setcommission(commission=commission_val/100)

    # Add a strategy
    if strategy == 'SMA':
        cerebro.addstrategy(SMAStrategy, maperiod=period, quantity=quantity, stopLoss=stopLoss, upper=upper, lower=lower)
    elif strategy == 'RSI':
        cerebro.addstrategy(RSIStrategy, maperiod=period, quantity=quantity, stopLoss=stopLoss, upper=upper, lower=lower)
    else :
        print('no strategy')
        exit()

    compression, timeframe = timeFrame(datapath)

    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname = datapath,
        dtformat = 2, 
        compression = compression, 
        timeframe = timeframe,
        fromdate = datetime.datetime.strptime(start, '%Y-%m-%d'),
        todate = datetime.datetime.strptime(end, '%Y-%m-%d'),
        reverse = False)


    # Add the Data Feed to Cerebro
    cerebro.adddata(data)


    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

#    try:     # convenience try/exception block
    if True:
        strat = cerebro.run()
        stratexe = strat[0]

        try:
            totalwin, totalloss, pnl_net = getWinLoss(stratexe.analyzers.ta.get_analysis())
        except KeyError:
            totalwin, totalloss, pnl_net = 0, 0, 0

        sqn = getSQN(stratexe.analyzers.sqn.get_analysis())

        if plt:
            cerebro.plot()

        return cerebro.broker.getvalue(), totalwin, totalloss, pnl_net, sqn

 #   except Exception as e:         # handle unexpected errors gracefully
 #       print('Error:', str(e))
 #       return 0, 0, 0, 0, 0


In [ ]:
!pip install backtest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement backtest (from versions: none)
ERROR: No matching distribution found for backtest


In [ ]:
import csv, os

commission_val = 0.0075 # spot% taker fees binance usdt futures
start_portfolio = 100000.0
portfolio = start_portfolio
stake_val = 1
quantity = 0.10

xchange = 'BTCUSDT'
start = '2021-01-01'
end = '2021-01-02'
strategy = 'RSI'
period = 12
stopLoss = 0
limits = [70,30]
subdir = 'data2'
plot = True
datapath = '/content/BTCUSDT-20210101-20210102-1m.csv'

print('\n ------------ ', datapath)
        
            
end_val, totalwin, totalloss, pnl_net, sqn = runbacktest(datapath, start, end, period, strategy, \
                                                                    limits[0], limits[1], commission_val, portfolio, stake_val, quantity, stopLoss, plot)

profit = (pnl_net / portfolio) * 100

# view the data in the console while processing
print('%s, %s (Pd %d) (SL %.1f%%) (U%d L%d) Net $%.2f (%.2f%%) WL %d/%d SQN %.2f' % (datapath[5:], strategy, period, stopLoss*100, limits[0], limits[1], end_val - start_portfolio, (end_val - start_portfolio)/start_portfolio*100, totalwin, totalloss, sqn))

result_writer.writerow([start, strategy, period, stopLoss*100, limits[0], limits[1], "{:.2f}".format(end_val - start_portfolio), totalwin, totalloss, sqn])


csvfile.close()


 ------------  /content/BTCUSDT-20210101-20210102-1m.csv
ent/BTCUSDT-20210101-20210102-1m.csv, RSI (Pd 12) (SL 0.0%) (U70 L30) Net $777.78 (0.78%) WL 18/7 SQN 1.76


In [ ]:
import csv, os
import time

commission_val = 0.0075 # spot% taker fees binance usdt futures
start_portfolio = 100000.0
portfolio = start_portfolio
stake_val = 1
quantity = 0.10

xchange = 'BTCUSDT'
start = '2021-01-01'
end = '2021-01-08'
strategies = ['RSI']
periodRange = range(12, 30)
stopLossRange = [0]
limits = [[70,30]]

#stopLossRange = [0, 0.001, 0.005, 0.01, -0.01]
#limits = [[70,30], [70,25], [60,25], [65,25], [70,20]]
#subdir = 'data2'
plot = False
datapath = 'BTCUSDT-20210101-20210102-1m.csv'


#dataname = 'result/{}-{}-{}-{}.csv'.format(xchange, start, end, '1m')
#csvfile = open(dataname, 'w', newline='', encoding='UTF8')
#result_writer = csv.writer(csvfile, delimiter=',')

startTime = time.time()

#for data in os.listdir("./"+subdir):
        
#datapath = subdir + '/' + data
#sep = datapath[len(subdir)+1:-4].split(sep='-') # ignore name file 'data/' and '.csv'
    # sep[0] = xchange  sep[1] = start  sep[2] = end  sep[3] = timeframe

print('\n ________________ ', datapath)
        
for strategy in strategies:

    for stopLoss in stopLossRange:

        for period in periodRange:

            for threshold in limits:
                
                end_val, totalwin, totalloss, pnl_net, sqn = runbacktest(datapath, start, end, period, strategy, \
                                                                                    threshold[0], threshold[1], commission_val, portfolio, stake_val, quantity, stopLoss, plot)

                profit = (pnl_net / portfolio) * 100

                # view the data in the console while processing
                print('%s, %s (Pd %d) (SL %.1f%%) (U%d L%d) Net $%.2f (%.2f%%) WL %d/%d SQN %.2f' % (datapath[5:], strategy, period, stopLoss*100, threshold[0], threshold[1], end_val - start_portfolio, (end_val - start_portfolio)/start_portfolio*100, totalwin, totalloss, sqn))

                #result_writer.writerow([sep[1], strategy, period, stopLoss*100, threshold[0], threshold[1], "{:.2f}".format(end_val - start_portfolio), totalwin, totalloss, sqn])


#csvfile.close()

endTime = time.time()
print()
print('elapsed secs:' + str(endTime - startTime))


 ________________  BTCUSDT-20210101-20210102-1m.csv
DT-20210101-20210102-1m.csv, RSI (Pd 12) (SL 0.0%) (U70 L30) Net $777.78 (0.78%) WL 18/7 SQN 1.76
DT-20210101-20210102-1m.csv, RSI (Pd 13) (SL 0.0%) (U70 L30) Net $189.57 (0.19%) WL 10/9 SQN 0.49
DT-20210101-20210102-1m.csv, RSI (Pd 14) (SL 0.0%) (U70 L30) Net $736.26 (0.74%) WL 10/6 SQN 1.72
DT-20210101-20210102-1m.csv, RSI (Pd 15) (SL 0.0%) (U70 L30) Net $701.66 (0.70%) WL 8/6 SQN 2.11
DT-20210101-20210102-1m.csv, RSI (Pd 16) (SL 0.0%) (U70 L30) Net $741.64 (0.74%) WL 11/3 SQN 2.15
DT-20210101-20210102-1m.csv, RSI (Pd 17) (SL 0.0%) (U70 L30) Net $596.69 (0.60%) WL 8/4 SQN 1.71
DT-20210101-20210102-1m.csv, RSI (Pd 18) (SL 0.0%) (U70 L30) Net $773.33 (0.77%) WL 8/3 SQN 1.93
DT-20210101-20210102-1m.csv, RSI (Pd 19) (SL 0.0%) (U70 L30) Net $762.12 (0.76%) WL 6/4 SQN 1.87
DT-20210101-20210102-1m.csv, RSI (Pd 20) (SL 0.0%) (U70 L30) Net $-21.51 (-0.02%) WL 2/3 SQN -0.05
DT-20210101-20210102-1m.csv, RSI (Pd 21) (SL 0.0%) (U70 L30) Net $-5